In [1]:
from azureml.core import Workspace, Experiment

#ws = Workspace.get(name="quick-starts-ws-153885")
ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-153885
Azure region: southcentralus
Subscription id: 6b4af8be-9931-443e-90f6-c4c34a1f9737
Resource group: aml-quickstarts-153885


In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# Create compute cluster
compute_cluster_name = "alpha"

try:
    compute_cluster = ComputeTarget(workspace=ws, name=compute_cluster_name)
    print("Found existing cluster, please use it.")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    compute_cluster = ComputeTarget.create(ws, compute_cluster_name, compute_config)

compute_cluster.wait_for_completion(show_output=True)

InProgress.....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [3]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling(
    {
        "--C": uniform(0.1, 1.0),
        "--max_iter": choice(25, 50, 100, 150)
    }
)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory=".", compute_target=compute_cluster, entry_script="train.py")

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                     hyperparameter_sampling=ps,
                                     policy = policy,
                                     primary_metric_name="Accuracy",
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

'SKLearn' estimator is deprecated. Please use 'ScriptRunConfig' from 'azureml.core.script_run_config' with your own defined environment or the AzureML-Tutorial curated environment.
'enabled' is deprecated. Please use the azureml.core.runconfig.DockerConfiguration object with the 'use_docker' param instead.


In [4]:
# Submit hyperdrive run to the experiment and show run details with the widget.

hdr = exp.submit(hyperdrive_config)
RunDetails(hdr).show()
hdr.wait_for_completion(show_output=True)

assert(hdr.get_status() == "Completed")

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d4628b90-0e0d-4602-b36f-903f7ea498ec
Web View: https://ml.azure.com/runs/HD_d4628b90-0e0d-4602-b36f-903f7ea498ec?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-153885/workspaces/quick-starts-ws-153885&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

"<START>[2021-08-07T08:21:35.165182][API][INFO]Experiment created<END>\n""<START>[2021-08-07T08:21:35.736579][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space<END>\n""<START>[2021-08-07T08:21:35.940367][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.<END>\n"

Execution Summary
RunId: HD_d4628b90-0e0d-4602-b36f-903f7ea498ec
Web View: https://ml.azure.com/runs/HD_d4628b90-0e0d-4602-b36f-903f7ea498ec?wsid=/subscriptions/6b4af8be-9931-443e-90f6-c4c34a1f9737/resourcegroups/aml-quickstarts-153885/workspaces/quick-starts-ws-153885&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254



In [5]:
import joblib
# Get best run and save the model from that run.

best_run_hdr = hdr.get_best_run_by_primary_metric()
best_run_metrics_hdr = best_run_hdr.get_metrics()
best_params_hdr = best_run_hdr.get_details()['runDefinition']['arguments']

print("\n")
print('Best run ID: ', best_run_hdr.id)
print('Best run Accuracy: ', best_run_metrics_hdr['Accuracy'])
print('Metrics: ', best_run_metrics_hdr)
print("\n")

print(best_run_hdr.get_file_names())

# Register model
#best_run_hdr.upload_file('outputs/best_model_hyperdrive.joblib', 'outputs/best_model_hyperdrive.joblib')
#model = best_run_hdr.register_model(model_name='best_model_hyperdrive', model_path='outputs/best_model_hyperdrive.joblib')
joblib.dump(best_params_hdr, filename='outputs/best_model_hyperdrive.joblib')



Best run ID:  HD_d4628b90-0e0d-4602-b36f-903f7ea498ec_2
Best run Accuracy:  0.9072837632776934
Metrics:  {'Regularization Strength:': 0.5056344015312062, 'Max iterations:': 25, 'Accuracy': 0.9072837632776934}


['azureml-logs/55_azureml-execution-tvmps_7ba94a4219893a3a77b878d30debf03ad339132b04bec3cfb818213b80f291e1_d.txt', 'azureml-logs/65_job_prep-tvmps_7ba94a4219893a3a77b878d30debf03ad339132b04bec3cfb818213b80f291e1_d.txt', 'azureml-logs/70_driver_log.txt', 'azureml-logs/75_job_post-tvmps_7ba94a4219893a3a77b878d30debf03ad339132b04bec3cfb818213b80f291e1_d.txt', 'azureml-logs/process_info.json', 'azureml-logs/process_status.json', 'logs/azureml/107_azureml.log', 'logs/azureml/job_prep_azureml.log', 'logs/azureml/job_release_azureml.log']


['outputs/best_model_hyperdrive.joblib']

In [6]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
data_path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"
ds = TabularDatasetFactory.from_delimited_files(path=data_path)

In [7]:
from train import clean_data
import pandas as pd

# Use the clean_data function to clean data
x, y = clean_data(ds)

In [8]:
from sklearn.model_selection import train_test_split
from azureml.core import Dataset
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train data with target
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv("training/train_data.csv", index=False)

# Store train data
data_store = ws.get_default_datastore()
data_store.upload(src_dir="./training", target_path="udacity-project", overwrite=True, show_progress=True)

# Reload train data
train_data = Dataset.Tabular.from_delimited_files(path=data_store.path("udacity-project/train_data.csv"))

Uploading an estimated of 1 files
Uploading ./training/train_data.csv
Uploaded ./training/train_data.csv, 1 files out of an estimated total of 1
Uploaded 1 files


In [9]:
from azureml.train.automl import AutoMLConfig

# Create AutoMLConfig specifying classification task and train data info and params
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='accuracy',
    training_data=train_data,
    label_column_name='y',
    n_cross_validations=4,
    compute_target=compute_cluster)

In [10]:
# Submit automl run

automl = exp.submit(automl_config)
RunDetails(automl).show()
automl.wait_for_completion(show_output=True)

assert(automl.get_status() == "Completed")

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5ab60472-18a9-4ec1-97c9-b43685d2a6dd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
udacity-project,AutoML_5ab60472-18a9-4ec1-97c9-b43685d2a6dd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetFeaturization. Beginning to fit featurizers and featurize the dataset.
Current status: DatasetBalancing. Performing class balancing sweeping
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       ALERTED
DESCRIPTION:  To decrease model bias, please cancel the current run and fix balancing problem.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData
DETAILS:      Imbalanced data can lead to a falsely perceived positive effect of a model's accuracy because the input data has bias towards one class.
+---------------------------------+---------------------------------+-------------------------

In [12]:
# Retrieve and save best automl model.

best_run_automl, best_model_automl = automl.get_output()
best_run_metrics_automl = best_run_automl.get_metrics()

print("\n")
print('Best run ID: ', best_run_automl.id)
print('Best run Accuracy: ', best_run_metrics_automl['accuracy'])
print('Metrics: ', best_run_metrics_automl)
print("\n")

# Register model
#best_run_automl.upload_file('outputs/best_model_automl.joblib', 'outputs/best_model_automl.joblib')
#model = best_run_automl.register_model(model_name='best_model_automl', model_path='outputs/best_model_automl.joblib')
joblib.dump(best_model_automl, filename='outputs/best_model_automl.joblib')



Best run ID:  AutoML_5ab60472-18a9-4ec1-97c9-b43685d2a6dd_29
Best run Accuracy:  0.9190440060698029
Metrics:  {'f1_score_macro': 0.7939297719210586, 'recall_score_micro': 0.9190440060698029, 'matthews_correlation': 0.5881065523806792, 'norm_macro_recall': 0.581082337069211, 'AUC_micro': 0.9808406538623611, 'precision_score_weighted': 0.9182078162705778, 'precision_score_macro': 0.7976971592815697, 'recall_score_weighted': 0.9190440060698029, 'AUC_weighted': 0.9481838936880631, 'recall_score_macro': 0.7905411685346055, 'balanced_accuracy': 0.7905411685346055, 'f1_score_weighted': 0.9185753244363074, 'f1_score_micro': 0.9190440060698029, 'weighted_accuracy': 0.9509449763133989, 'log_loss': 0.22503428584436114, 'average_precision_score_weighted': 0.9560620267940345, 'AUC_macro': 0.948183893688063, 'accuracy': 0.9190440060698029, 'average_precision_score_micro': 0.9815596471253963, 'average_precision_score_macro': 0.8271358405542905, 'precision_score_micro': 0.9190440060698029, 'confusio

['outputs/best_model_automl.joblib']

In [13]:
# Print best model info
print(best_run_automl)
print("\n")
print(best_model_automl)

Run(Experiment: udacity-project,
Id: AutoML_5ab60472-18a9-4ec1-97c9-b43685d2a6dd_29,
Type: azureml.scriptrun,
Status: Completed)


Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
), random_state=None, reg_alpha=0.3157894736842105, reg_lambda=0.8421052631578947, subsample=1))], verbose=False)), ('13', Pipeline(memory=None, steps=[('maxabsscaler', MaxAbsScaler(copy=True)), ('sgdclassifierwrapper', SGDClassifierWrapper(alpha=7.5510448979591835, class_weight='balanced', eta0=0.001, fit_intercept=True, l1_ratio=0.42857142857142855, learning_rate='constant', loss='modified_huber', max_iter=1000, n_jobs=1, penalty='none', power_t=0.7777777777777777, rando

In [14]:
# Cleanup cluster
compute_cluster.delete()

Current provisioning state of AmlCompute is "Deleting"

